## This notebook is used to run hyperparameter optimization studies with Optuna.

In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/sulfur_tes_ml/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import train_and_validate_model

/Users/kmenear/Projects/sulfur_tes_ml/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Define parameters used for model training in the objective function below.

In [3]:
data_dir = "../data/Sulfur_Models/heating/full_runs"
model_type = 'NN' # Options: NN, XGBoost, RandomForest
target = 'Tavg' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.
split_test_data = False # Split data into train (64%), val (16%), and test (20%) (True) or just train (80%) and val (20%) (False)

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'

### Define the objective function.
See https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/001_first.html#sphx-glr-tutorial-10-key-features-001-first-py for an introduction to using Optuna for hyperparameter optimization.

In [4]:
def objective(trial):
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        learning_rate = trial.suggest_float("learning_rate", 0.00001, 0.5, log=True)
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 1000)
        batch_size = trial.suggest_int("batch_size", 100, 10000)
        parameters = {'learning_rate': learning_rate, 'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, split_test_data=split_test_data)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.1, 0.5)
        subsample = trial.suggest_float("subsample", 0.01, 1)
        max_depth = trial.suggest_int("max_depth", 1, 20)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'max_depth': max_depth, 'num_boost_round': num_boost_round}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, split_test_data=split_test_data, t_min=360)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, split_test_data=split_test_data)
    
    return result

### If continuing a previous study, set `load_study` to `True` and provide the study name.
*Note:* Studies are saved in the `studies` directory.

In [6]:
load_study = False
study_name = "study_name_.pkl"

### Either load the previous study or create a new study.

In [7]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

[I 2022-09-21 00:47:56,831] A new study created in memory with name: no-name-6e1f30a8-6bfb-4214-ab51-89194b9a7ac7


### Run the study.

In [8]:
study.optimize(objective, n_trials=250)

2022-09-21 00:48:35.293263: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 00:48:35.399598: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/200
179/665 [=======>......................] - ETA: 7s - loss: 0.1693


KeyboardInterrupt



### Print the hyperparameter and objective function values for the best trial in the study.

In [ ]:
print(study.best_params)

In [21]:
print(study.best_value)